In [1]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
# Embedding setup
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [4]:
# CNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
cnn = keras.Model(int_sequences_input, preds)
cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [5]:
# LSTM setup

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(4,input_shape=(1,5))(embedded_sequences)
out = layers.Dense(1,activation='relu')(x)

lstm = keras.Model(int_sequences_input,out)
lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [6]:
# RNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.GRU(256,return_sequences=True)(embedded_sequences)
x = layers.SimpleRNN(128)(x)
out = layers.Dense(1,activation='relu')(x)

rnn = keras.Model(int_sequences_input,out)
rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [7]:
# Data setup
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

results = pd.DataFrame()

In [8]:
# Try CNN with batch sizes 10,32 and epochs 20,50,75 and 100
#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["CNN 20 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["CNN E20 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["CNN E100 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["CNN E100 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["CNN E75 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["CNN E75 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["CNN E50 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["CNN E50 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 4s 6ms/step - loss: 1.9418 - mean_absolute_error: 1.0629 - mean_absolute_percentage_error: 28.1645
Epoch 2/20
563/563 [==============================] - 4s 7ms/step - loss: 1.0161 - mean_absolute_error: 0.8062 - mean_absolute_percentage_error: 21.9172
Epoch 3/20
563/563 [==============================] - 5s 8ms/step - loss: 0.9621 - mean_absolute_error: 0.7848 - mean_absolute_percentage_error: 21.3010
Epoch 4/20
563/563 [==============================] - 5s 8ms/step - loss: 0.9260 - mean_absolute_error: 0.7696 - mean_absolute_percentage_error: 20.8585
Epoch 5/20
563/563 [==============================] - 4s 8ms/step - loss: 0.9115 - mean_absolute_error: 0.7679 - mean_absolute_percentage_error: 20.6816
Epoch 6/20
563/563 [==============================] - 5s 9ms/step - loss: 0.8580 - mean_absolute_error: 0.7428 - mean_absolute_percentage_error: 20.0870
Epoch 7/20
563/563 [==============================] - 5s 8ms/step - loss: 0.8271 -

In [9]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32
0,0.652319,0.679281,0.802386,0.816223,0.804668,0.792130,0.807721,0.769787
1,0.643356,0.655891,0.708887,0.718937,0.716400,0.711506,0.715247,0.699274
2,17.783934,18.514244,19.542551,19.893368,19.647087,19.806202,19.740183,19.594261


In [10]:
# Try LSTM with batch sizes 10,32 and epochs 20,50,75 and 100
#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["LSTM E20 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["LSTM E20 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["LSTM E100 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["LSTM E100 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["LSTM E75 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["LSTM E75 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["LSTM E50 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["LSTM E50 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 19s 33ms/step - loss: 15.7647 - mean_absolute_error: 3.7996 - mean_absolute_percentage_error: 92.6072
Epoch 2/20
563/563 [==============================] - 19s 33ms/step - loss: 8.9993 - mean_absolute_error: 2.7722 - mean_absolute_percentage_error: 64.8659
Epoch 3/20
563/563 [==============================] - 19s 33ms/step - loss: 6.0523 - mean_absolute_error: 2.1972 - mean_absolute_percentage_error: 49.7865
Epoch 4/20
563/563 [==============================] - 18s 33ms/step - loss: 4.1532 - mean_absolute_error: 1.7580 - mean_absolute_percentage_error: 39.3996
Epoch 5/20
563/563 [==============================] - 18s 33ms/step - loss: 2.7506 - mean_absolute_error: 1.3946 - mean_absolute_percentage_error: 31.8987
Epoch 6/20
563/563 [==============================] - 19s 33ms/step - loss: 2.0830 - mean_absolute_error: 1.2072 - mean_absolute_percentage_error: 28.7855
Epoch 7/20
563/563 [==============================] - 19s 33ms/step -

In [11]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32,LSTM E20 B10,LSTM E20 B32,LSTM E100 B10,LSTM E100 B32,LSTM E75 B10,LSTM E75 B32,LSTM E50 B10,LSTM E50 B32
0,0.652319,0.679281,0.802386,0.816223,0.804668,0.792130,0.807721,0.769787,1.315162,1.315397,1.315131,1.315246,1.316236,1.314948,1.314834,1.315323
1,0.643356,0.655891,0.708887,0.718937,0.716400,0.711506,0.715247,0.699274,0.957143,0.957119,0.957147,0.957134,0.957038,0.957167,0.957180,0.957126
2,17.783934,18.514244,19.542551,19.893368,19.647087,19.806202,19.740183,19.594261,29.671551,29.688011,29.669312,29.677523,29.741392,29.655855,29.647215,29.682915


In [12]:
# Try RNN with batch sizes 10,32 and epochs 50,75 and 100
#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["RNN E100 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["RNN E100 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["RNN E75 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["RNN E75 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["RNN E50 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["RNN E50 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/100
563/563 [==============================] - 70s 123ms/step - loss: 2.1436 - mean_absolute_error: 1.1449 - mean_absolute_percentage_error: 31.5560
Epoch 2/100
563/563 [==============================] - 69s 123ms/step - loss: 1.2755 - mean_absolute_error: 0.9447 - mean_absolute_percentage_error: 28.1254
Epoch 3/100
563/563 [==============================] - 66s 116ms/step - loss: 1.2728 - mean_absolute_error: 0.9437 - mean_absolute_percentage_error: 28.0524
Epoch 4/100
563/563 [==============================] - 71s 126ms/step - loss: 1.2759 - mean_absolute_error: 0.9445 - mean_absolute_percentage_error: 28.1128
Epoch 5/100
563/563 [==============================] - 67s 119ms/step - loss: 1.2750 - mean_absolute_error: 0.9436 - mean_absolute_percentage_error: 28.0573
Epoch 6/100
563/563 [==============================] - 68s 121ms/step - loss: 1.2800 - mean_absolute_error: 0.9459 - mean_absolute_percentage_error: 28.1533
Epoch 7/100
563/563 [==============================] - 68s

In [17]:
results = results.T
#results.columns = ["MSE","MAE","% Error"]
results.to_csv("../../data/20_50_75_100_epochs_test.csv")